In [11]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Load an image for prediction (for example, from a file)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import cv2

import numpy as np
import mediapipe as mp

In [6]:

# Image settings
img_height, img_width = 224, 224  # You said 224 earlier
batch_size = 32

# Paths
train_path = '../Datasets/Eye_Direction/TrainingSet/TrainingSet'
test_path = '../Datasets/Eye_Direction/TestSet/TestSet'

# 1. Create ImageDataGenerators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Important: set 20% aside for validation
)

test_datagen = ImageDataGenerator(rescale=1./255)

# 2. Create Data Generators

# Training generator (80% of training set)
train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # Multi-class classification
    shuffle=True,
    subset='training',  # Take only training data
    seed=123
)

# Validation generator (20% of training set)
val_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation',  # Take only validation data
    seed=123
)

# Testing generator (no shuffle here)
test_gen = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 106106 images belonging to 8 classes.
Found 26524 images belonging to 8 classes.
Found 61073 images belonging to 8 classes.


In [8]:
model = Sequential([
    # Feature Extraction
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Classification Head
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_gen.class_indices), activation='softmax')  # Dynamic class count
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,246,728 (84.86 MB)

 Trainable params: 22,246,728 (84.86 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
# 3. Callbacks (Optional but Recommended)
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)
]

In [14]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
3316/3316 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8057 - loss: 0.4912

3316/3316 ━━━━━━━━━━━━━━━━━━━━ 4888s 1s/step - accuracy: 0.8057 - loss: 0.4912 - val_accuracy: 0.9379 - val_loss: 0.1568 - learning_rate: 0.0010
Epoch 2/10
3316/3316 ━━━━━━━━━━━━━━━━━━━━ 0s 564ms/step - accuracy: 0.9291 - loss: 0.1809

3316/3316 ━━━━━━━━━━━━━━━━━━━━ 2114s 637ms/step - accuracy: 0.9291 - loss: 0.1809 - val_accuracy: 0.9492 - val_loss: 0.1198 - learning_rate: 0.0010
Epoch 3/10
3316/3316 ━━━━━━━━━━━━━━━━━━━━ 0s 564ms/step - accuracy: 0.9393 - loss: 0.1526

3316/3316 ━━━━━━━━━━━━━━━━━━━━ 2114s 638ms/step - accuracy: 0.9393 - loss: 0.1526 - val_accuracy: 0.9542 - val_loss: 0.1080 - learning_rate: 0.0010
Epoch 4/10
3316/3316 ━━━━━━━━━━━━━━━━━━━━ 0s 565ms/step - accuracy: 0.9447 - loss: 0.1347

3316/3316 ━━━━━━━━━━━━━━━━━━━━ 2112s 637ms/step - accuracy: 0.9447 - loss: 0.1347 - val_accuracy: 0.9551 - val_loss: 0.1092 - learning_rate: 0.0010
Epoch 5/10
3316/3316 ━━━━━━━━━━━━━━━━━━━━ 0s 563ms/step - accuracy: 0.9503 - loss: 0.1221

3316/3316 ━━━━━━━━━━━━━━━━━━━━ 2108s 636ms/step - accuracy: 0.9503 - loss: 0.1221 - val_accuracy: 0.9600 - val_loss: 0.0950 - learning_rate: 0.0010
Epoch 6/10
3316/3316 ━━━━━━━━━━━━━━━━━━━━ 0s 563ms/step - accuracy: 0.9523 - loss: 0.1167

3316/3316 ━━━━━━━━━━━━━━━━━━━━ 2110s 636ms/step - accuracy: 0.9523 - loss: 0.1167 - val_accuracy: 0.9618 - val_loss: 0.0913 - learning_rate: 0.0010
Epoch 7/10
3316/3316 ━━━━━━━━━━━━━━━━━━━━ 2102s 634ms/step - accuracy: 0.9539 - loss: 0.1123 - val_accuracy: 0.9582 - val_loss: 0.1085 - learning_rate: 0.0010
Epoch 8/10
3316/3316 ━━━━━━━━━━━━━━━━━━━━ 0s 562ms/step - accuracy: 0.9567 - loss: 0.1073

3316/3316 ━━━━━━━━━━━━━━━━━━━━ 2101s 634ms/step - accuracy: 0.9567 - loss: 0.1073 - val_accuracy: 0.9628 - val_loss: 0.0876 - learning_rate: 0.0010
Epoch 9/10
3316/3316 ━━━━━━━━━━━━━━━━━━━━ 0s 563ms/step - accuracy: 0.9583 - loss: 0.1026

3316/3316 ━━━━━━━━━━━━━━━━━━━━ 2106s 635ms/step - accuracy: 0.9583 - loss: 0.1026 - val_accuracy: 0.9639 - val_loss: 0.0843 - learning_rate: 0.0010
Epoch 10/10
3316/3316 ━━━━━━━━━━━━━━━━━━━━ 2099s 633ms/step - accuracy: 0.9581 - loss: 0.1041 - val_accuracy: 0.9615 - val_loss: 0.0910 - learning_rate: 0.0010


In [16]:
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy:.2f}")

1909/1909 ━━━━━━━━━━━━━━━━━━━━ 559s 293ms/step - accuracy: 0.9637 - loss: 0.0880
Test Accuracy: 0.97


In [ ]:
model.save("models/eyes_dir_model.h5")